🔁 Big picture first

This output is a chronological message log produced by AutoGen showing:

User sends a question

Assistant decides to call a tool

Tool executes

Assistant summarizes tool output into a normal message

Think of it as AutoGen’s internal audit trail.


1️⃣ TextMessage — User input

In [ ]:
TextMessage(
  source='user',
  content='Find information on AutoGen',
  created_at=2026-02-07 08:04:26 UTC,
  type='TextMessage'
)


What this means

This is the initial human input

Stored as a structured object, not plain text

models_usage=None → no model was invoked yet

💡 AutoGen treats everything (human, agent, tool output) as messages in the same timeline.

2️⃣ ToolCallRequestEvent — Assistant decides to use a tool

In [ ]:
ToolCallRequestEvent(
  source='assistant',
  models_usage=RequestUsage(prompt_tokens=61, completion_tokens=15),
  content=[
    FunctionCall(
      name='web_search',
      arguments='{"query":"AutoGen"}'
    )
  ],
  type='ToolCallRequestEvent'
)


What happened here

The assistant reasoned:

“I don’t have enough guaranteed info → I should call a tool”

So it:

Selected the web_search tool

Passed structured arguments ({"query": "AutoGen"})

Emitted a tool request event, not a final answer

Important details

models_usage is present → this used an LLM step

This step is planning, not answering

No text response yet — just tool intent

💡 This is AutoGen’s ReAct-style behavior: Reason → Act

3️⃣ ToolCallExecutionEvent — Tool actually runs

In [ ]:
ToolCallExecutionEvent(
  source='assistant',
  content=[
    FunctionExecutionResult(
      name='web_search',
      content='AutoGen is a programming framework for building multi-agent applications.',
      is_error=False
    )
  ],
  type='ToolCallExecutionEvent'
)


What this means

The runtime (not the LLM) executed the tool

Tool returned raw output

No model tokens used here (models_usage=None)

Key distinction
Step	Who does it
ToolCallRequestEvent	LLM
ToolCallExecutionEvent	Tool runtime

💡 This separation is why AutoGen can swap tools, sandboxes, or even humans-in-the-loop.

4️⃣ ToolCallSummaryMessage — Assistant converts tool output into a message

In [ ]:
ToolCallSummaryMessage(
  source='assistant',
  content='AutoGen is a programming framework for building multi-agent applications.',
  tool_calls=[...],
  results=[...],
  type='ToolCallSummaryMessage'
)


What happened here

The assistant:

Took the tool result

Summarized it into human-readable content

Attached:

tool_calls → what was invoked

results → what came back

This is the bridge between tools and conversation.

💡 This message is what downstream agents or the UI usually consume.

🔎 Mental model (super important)

Think of AutoGen logs as an event-sourced system:


User Message

   ↓

Agent Planning (LLM)

   ↓

Tool Invocation

   ↓

Tool Execution

   ↓

Agent Interpretation